In [18]:
import os
import numpy as np
import pandas as pd
import csv
import gzip
import json

In [10]:
path = "/mnt/L3MVN/data/matterport_category_mappings.tsv"

df = pd.read_csv(path, sep="\t")
print(df.head())

  index    raw_category    category    count    nyuId    nyu40id    eigen13id    nyuClass    nyu40class    eigen13class    ModelNet40    ModelNet10    ShapeNetCore55    synsetoffset    wnsynsetid    wnsynsetkey    mpcat40index    mpcat40
0  1    wall    wall    7667    21    1    12    ...                                                                                                                                                                                         
1  2    door    door    2544    28    8    12    ...                                                                                                                                                                                         
2  3    ceiling    ceiling    2363    4    22    ...                                                                                                                                                                                         
3  4    floor    floor    2252    11    2    5  

In [86]:
items = []

with open(path, 'r') as f:
    text = f.read()
lines = text.split('\n')

for l in lines:
    items.append(l.split('    '))

items_filt = [r for r in items if len(r) > 3]

In [34]:
ind = 0

print(items[ind][2], items[ind][-1])

category mpcat40


In [37]:
cats = np.unique([r[2] for r in items if len(r)>3])
cats_mp40 = np.unique([r[-1] for r in items if len(r)>3])

print(len(cats), len(cats_mp40))

1147 43


In [38]:
cats_mp40

array(['appliances', 'bathtub', 'beam', 'bed', 'blinds', 'board_panel',
       'cabinet', 'ceiling', 'chair', 'chest_of_drawers', 'clothes',
       'column', 'counter', 'curtain', 'cushion', 'door', 'fireplace',
       'floor', 'furniture', 'gym_equipment', 'lighting', 'mirror',
       'misc', 'mpcat40', 'objects', 'picture', 'plant', 'railing',
       'seating', 'shelving', 'shower', 'sink', 'sofa', 'stairs', 'stool',
       'table', 'toilet', 'towel', 'tv_monitor', 'unlabeled', 'void',
       'wall', 'window'], dtype='<U16')

In [23]:
scene_info_path = "data/objectgoal_hm3d/val/content/4ok3usBNeis.json.gz"
scene_info_path = "data/objectgoal_PersONAL/val/easy/content/4ok3usBNeis.json.gz"

with gzip.open(scene_info_path, "r") as f:
    info = json.load(f)

info.keys()

dict_keys(['goals_by_category', 'episodes', 'category_to_task_category_id', 'category_to_scene_annotation_category_id'])

In [24]:
info["episodes"][0]

{'episode_id': '195',
 'scene_id': 'hm3d_v0.2/val/00877-4ok3usBNeis/4ok3usBNeis.basis.glb',
 'scene_dataset_config': './data/scene_datasets/hm3d_v0.2/hm3d_annotated_basis.scene_dataset_config.json',
 'object_category': 'couch',
 'object_id': 'couch_160',
 'description': ['plaid couch in the room. it is of a red-brown color.',
  '',
  ''],
 'owner': 'Sebastian',
 'floor_id': '0',
 'summary': "On the ground floor, there is a plaid couch in the living room, distinguished by its red-brown color. This couch belongs to Sebastian. In the bathroom on the same floor, a washbasin is situated near the toilet seat and the mirror, and it is owned by Wesley. Each person's belongings add character to their respective rooms.",
 'extracted_summary': ['Sebastian owns a plaid red-brown couch in the living room',
  'Wesley owns a washbasin near the toilet seat and the mirror in the bathroom'],
 'query': ["Find Sebastian's couch",
  "Where is Sebastian's couch?",
  "Locate Sebastian's couch",
  "Retrieve S

In [100]:
personal_to_mp3d = {}
personal_counts = {}

scene_info_dir = os.path.dirname(scene_info_path)

for scene_path in os.listdir(scene_info_dir):

    scene_path = os.path.join(scene_info_dir, scene_path)
    with gzip.open(scene_path, "r") as f:
        scene_info = json.load(f)

    for ep in scene_info["episodes"]:

        obj_cat = ep["object_category"]

        cat_row = next(r for r in items_filt if r[2] == obj_cat)
        obj_cat_map = cat_row[-1]
        obj_cat_id = int(cat_row[-2]) + 1

        obj_cat_map = f"{obj_cat_map}_{str(obj_cat_id)}"
        
        # cat_map = [r[-1] for r in items_filt if r[2] == obj_cat]
        # assert len(cat_map)>0 and len(np.unique(cat_map))==1

        # obj_cat_map = cat_map[0]

        personal_to_mp3d[obj_cat] = obj_cat_map

        if obj_cat_map in personal_counts:
            personal_counts[obj_cat_map] += 1
        else:
            personal_counts[obj_cat_map] = 1

In [101]:
personal_to_mp3d

{'sideboard': 'misc_41',
 'toilet': 'toilet_19',
 'blanket': 'objects_40',
 'table': 'table_6',
 'washer-dryer': 'misc_41',
 'tv': 'tv_monitor_23',
 'sink cabinet': 'cabinet_8',
 'mantle': 'fireplace_28',
 'bed': 'bed_12',
 'couch': 'sofa_11',
 'washing machine': 'appliances_38',
 'sofa seat': 'misc_41',
 'kitchen shelf': 'shelving_32',
 'sofa chair': 'chair_4',
 'bathtub': 'bathtub_26',
 'sofa set': 'sofa_11',
 'clothes': 'clothes_39',
 'bathroom cabinet': 'cabinet_8',
 'shower cabin': 'misc_41',
 'towel': 'towel_21',
 'stool': 'stool_20',
 'oven': 'appliances_38',
 'chair': 'chair_4',
 'wardrobe': 'furniture_37',
 'sink': 'sink_16',
 'stove': 'appliances_38',
 'clothes dryer': 'appliances_38',
 'box': 'objects_40',
 'drawer': 'chest_of_drawers_14',
 'desk': 'table_6',
 'crib': 'misc_41',
 'oven and stove': 'misc_41',
 'balustrade': 'railing_31',
 'bench': 'seating_35',
 'office chair': 'chair_4',
 'handbag': 'objects_40',
 'bathroom towel': 'towel_21',
 'plush toy': 'objects_40',
 'h

In [102]:
print(len(personal_to_mp3d.keys()), len(np.unique(list(personal_to_mp3d.values()))))

58 23


In [103]:
personal_to_mp3d.values()

dict_values(['misc_41', 'toilet_19', 'objects_40', 'table_6', 'misc_41', 'tv_monitor_23', 'cabinet_8', 'fireplace_28', 'bed_12', 'sofa_11', 'appliances_38', 'misc_41', 'shelving_32', 'chair_4', 'bathtub_26', 'sofa_11', 'clothes_39', 'cabinet_8', 'misc_41', 'towel_21', 'stool_20', 'appliances_38', 'chair_4', 'furniture_37', 'sink_16', 'appliances_38', 'appliances_38', 'objects_40', 'chest_of_drawers_14', 'table_6', 'misc_41', 'misc_41', 'railing_31', 'seating_35', 'chair_4', 'objects_40', 'towel_21', 'objects_40', 'objects_40', 'objects_40', 'towel_21', 'misc_41', 'cabinet_8', 'appliances_38', 'shelving_32', 'objects_40', 'sink_16', 'shower_24', 'misc_41', 'clothes_39', 'board_panel_36', 'shelving_32', 'cabinet_8', 'misc_41', 'misc_41', 'tv_monitor_23', 'table_6', 'objects_40'])

In [104]:
np.unique(list(personal_to_mp3d.values()))

array(['appliances_38', 'bathtub_26', 'bed_12', 'board_panel_36',
       'cabinet_8', 'chair_4', 'chest_of_drawers_14', 'clothes_39',
       'fireplace_28', 'furniture_37', 'misc_41', 'objects_40',
       'railing_31', 'seating_35', 'shelving_32', 'shower_24', 'sink_16',
       'sofa_11', 'stool_20', 'table_6', 'toilet_19', 'towel_21',
       'tv_monitor_23'], dtype='<U19')

In [105]:
personal_counts

{'misc_41': 34,
 'toilet_19': 30,
 'objects_40': 43,
 'table_6': 67,
 'tv_monitor_23': 54,
 'cabinet_8': 35,
 'fireplace_28': 1,
 'bed_12': 61,
 'sofa_11': 46,
 'appliances_38': 35,
 'shelving_32': 7,
 'chair_4': 62,
 'bathtub_26': 18,
 'clothes_39': 13,
 'towel_21': 22,
 'stool_20': 4,
 'furniture_37': 20,
 'sink_16': 28,
 'chest_of_drawers_14': 6,
 'railing_31': 4,
 'seating_35': 4,
 'shower_24': 2,
 'board_panel_36': 4}

In [106]:
sum(personal_counts.values())

600

In [107]:
items_filt[0]

['index',
 'raw_category',
 'category',
 'count',
 'nyuId',
 'nyu40id',
 'eigen13id',
 'nyuClass',
 'nyu40class',
 'eigen13class',
 'ModelNet40',
 'ModelNet10',
 'ShapeNetCore55',
 'synsetoffset',
 'wnsynsetid',
 'wnsynsetkey',
 'mpcat40index',
 'mpcat40']

In [108]:
mp_cat_ids = [4, 11, 15, 12, 19, 23, 26, 24, 28, 38, 21, 16, 14, 6, 16]
mp_cat_names = []

for id in mp_cat_ids:
    id_row = next(r for r in items_filt if r[16] == str(id-1))
    mp_cat_names.append(id_row[-1])

mp_cat_names

['chair',
 'sofa',
 'plant',
 'bed',
 'toilet',
 'tv_monitor',
 'bathtub',
 'shower',
 'fireplace',
 'appliances',
 'towel',
 'sink',
 'chest_of_drawers',
 'table',
 'sink']

In [109]:
filt_counts = {}

for k, v in personal_counts.items():

    id = int( k.split("_")[-1] )
    if id in mp_cat_ids:
        filt_counts[k] = v


print(f"{sum(filt_counts.values())}/{sum(personal_counts.values())}\n")

filt_counts

432/600



{'toilet_19': 30,
 'table_6': 67,
 'tv_monitor_23': 54,
 'fireplace_28': 1,
 'bed_12': 61,
 'sofa_11': 46,
 'appliances_38': 35,
 'chair_4': 62,
 'bathtub_26': 18,
 'towel_21': 22,
 'sink_16': 28,
 'chest_of_drawers_14': 6,
 'shower_24': 2}